<a href="https://colab.research.google.com/github/DanielKorenDataScience/NLP-FinalProject/blob/main/ChatBotBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from importlib import import_module
def my_import(lib, package=None, func=None):
  try:
    import_module(lib)
  except:
    if package is None: package = lib
    !pip install "{package}"
    import_module(lib)
  
  if func is not None:
    return import_module(lib).__getattribute__(func)

In [ ]:
my_import('pyforest')
import json
import string
import random
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

p = print
d = display

  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14606 sha256=fb9f8b6f55ba63c58f24fb3a0373b2099107fcca520e078ff248093057ab2c9a
  Stored in directory: /root/.cache/pip/wheels/40/c6/da/43562aeea85b37f1a2b3d326f0f602f865000d2ada8a43625f
Successfully built pyforest


In [ ]:
my_import('spacy')
from spacy.tokens import Doc, Span
from spacy.matcher import PhraseMatcher

# Load md (medium) English tokenizer, tagger, parser and NER, and vectors(vectors are not included in the 'sm' model)
#nlp = spacy.load('en_core_web_md')  NOT WORKING ON COLAB
try:
  import en_core_web_md
except:  
  !python -m spacy download en_core_web_md
  import en_core_web_md
  
nlp = en_core_web_md.load()

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp37-none-any.whl size=98051304 sha256=ab6fdd9ceab8633253d73ab8826da841ace403e252d921b9864abb3058c8d757
  Stored in directory: /tmp/pip-ephem-wheel-cache-9hq_bhzu/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import requests

api_key = 'c2adfa29edfd95ad16efab9218619ff3'
URL = "http://api.openweathermap.org/data/2.5/weather?"

In [ ]:
path = '/content/drive/MyDrive/Project5_ChatBot/'
data_url = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/data/' 

In [ ]:
TEST = True

# Call the web site

{"coord":{"lon":34.8,"lat":32.0833},"weather":[{"id":801,"main":"Clouds","description":"few clouds","icon":"02n"}],"base":"stations","main":{"temp":295.69,"feels_like":295.83,"temp_min":294.42,"temp_max":296.58,"pressure":1016,"humidity":70},"visibility":10000,"wind":{"speed":2.06,"deg":10},"clouds":{"all":20},"dt":1623961727,"sys":{"type":1,"id":6845,"country":"IL","sunrise":1623897263,"sunset":1623948546},"timezone":10800,"id":293396,"name":"Tel Aviv","cod":200}

In [ ]:
WEATHER_ONLY = 1

In [ ]:
def call_web(city_name, country, count=1):
    state = ''
    api_url = f"{URL}q={city_name},{state},{country}&cnt={count}&appid={api_key}"
    #p(api_url)

    response = requests.get(api_url)
    response_dict = response.json()

    weather = response_dict #["weather"]#[0]#["description"]

    if response.status_code == 200:
        return weather
    elif response.status_code == 404:
        return None
    else:
        print('[!] HTTP {0} calling [{1}]'.format(response.status_code, api_url))
        return None

In [ ]:
def get_weather(dt, location, action = None):
  template = WEATHER_ONLY

  city_w = city = location[0][0]  # assume one city (not handling ifthere are 2 cities)
  #p(city, city_w)
  result = ''
  # check if city in api_city list
  if city in df_CITIES_API.index.values:
    result = df_CITIES_API.loc[city]
  if len(result) == 0: # not found, get district
    if city in cities_il_df.index.values:
      city_w = cities_il_df.loc[city].district
  elif len(result) > 0: # more than one city -> need country
    pass

  #p(city_w)
  country = location[0][1]
  country_code = countries_df[countries_df.Name == country].index[0]
  
  city_weather = call_web(city_w, country_code)
  if city_weather is not None:
    if template == WEATHER_ONLY:
      result = city_weather["weather"][0]["description"]
    return f"In {city.capitalize()}, {location[0][1].capitalize()} the current weather is: {result}"
  else:
    return "Something went wrong."
 
#answer = get_weather('current', [('toronto', 'united states')])
#answer

# Get cities and update model

In [ ]:
GIT = True
git_path = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/data/'

def get_json(fname, GIT=True):
  p(fname)
  if GIT:
    resp = requests.get(data_url + fname)
    return json.loads(resp.text)
  with open(path + fname, encoding="utf8") as f:
    return json.loads(f.read())

In [ ]:
intents = get_json('intents.json')
#intents

intents.json


In [ ]:
def load_CITIES_API():
  # file too big for git so read it from my google drive using a shared link
  #CITIES_API = get_json('city.list.json')
  git_path = 'https://raw.githubusercontent.com/DanielKorenDataScience/NLP-FinalProject/main/' + 'city.list.json'
  resp = requests.get(git_path)
  CITIES_API = json.loads(resp.text)

  ''' 
  google_shared_link = 'https://drive.google.com/file/d/19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT/view?usp=sharing'
  url = 'https://drive.google.com/uc?authuser=0&id=19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT&export=download'
  file_id = '19lB4T32o1VbFAKRk_PExbNYV8mcZ8hRT'
  
  while True:
    try:
      with open('city.list.json', encoding="utf8") as f:
        CITIES_API = json.loads(f.read())
      break
    except:
      wget.download(url)  # download the file to local google drive with the original name in the share link
      # now can open with no path because it is in the local 'colab' directory
  
  '''
  df = pd.DataFrame(CITIES_API)
  df = df.drop(['id', 'coord'], axis='columns')
  df.name = df.name.str.lower()
  df.name = df.name.str.translate(remove_punct_dict)

  CITIES_API_city = sorted(set(df.name.to_list()))
  CITIES_API_country = sorted(set(df.country.str.lower().to_list()))
  if CITIES_API_country[0] == '':
    CITIES_API_country = CITIES_API_country[1:]
  CITIES_API_state = sorted(set(df.state.str.lower().to_list()))
  if CITIES_API_state[0] == '':
    CITIES_API_state = CITIES_API_state[1:]
  if CITIES_API_state[0] == '00':
    CITIES_API_state = CITIES_API_state[1:]
  
  df.set_index('name', inplace=True)
  #d(df.head)
  return df, CITIES_API_city, CITIES_API_country, CITIES_API_state

df_CITIES_API, CITIES_API_city, CITIES_API_country_code, CITIES_API_state_code = load_CITIES_API()
df_CITIES_API.head()

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=5587168e964b6dad02dfdf6b07c68367c2a752a48d0dae959dfdef629f2f555e
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


<IPython.core.display.Javascript object>

,state,country
name,,
ḩeşāre sefīd,,IR
‘ayn ḩalāqīm,,SY
taglag,,IR
qabāghlū,,IR
‘arīqah,,SY


In [ ]:
def load_countries():
  path_ = path  
  fname = 'countries.csv'
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname)
  df.Name = df.Name.str.lower()
  COUNTRIES = df.Name.tolist()
  df = df.set_index('Code')
  return df, COUNTRIES

countries_df, COUNTRIES = load_countries()
countries_df.head()

<IPython.core.display.Javascript object>

,Name
Code,
AF,afghanistan
AX,åland islands
AL,albania
DZ,algeria
AS,american samoa


In [ ]:
def load_cities_il():
  path_ = path
  fname = 'cities_IL.csv'
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname, names=['city', 'district'])
  df.city = df.city.str.translate(remove_punct_dict)
  CITIES = df.city.tolist()
  df = df.set_index('city')
  return df, CITIES
cities_il_df, CITIES_IL = load_cities_il()
cities_il_df.head()

<IPython.core.display.Javascript object>

,district
city,
asam,beersheba
abbirim,acre
abu abdun,beersheba
abu ammar,beersheba
abu amre,beersheba


In [ ]:
def load_capitals():
  path_ = path
  fname = 'capitals.json'
  if GIT:
    path_ = git_path
  CAPITALS = pd.read_json(path_ + fname, encoding="utf8", typ='series').str.lower()

  CAPITALS.index = CAPITALS.index.str.lower()
  df = pd.DataFrame([CAPITALS])
  df = df.T
  df = df.reset_index()
  df.columns = ['country', 'capital']
  df = df.set_index('capital')
  return df, CAPITALS.to_dict()
capitals_df, CAPITALS = load_capitals()

p(CAPITALS['albania'])
p(capitals_df.loc['madrid'].country)
capitals_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

tirana
spain


,country
capital,
kabul,afghanistan
mariehamn,åland islands
tirana,albania
algiers,algeria
pago pago,american samoa


In [ ]:
def load_largest():
  fname = '100-largest-cities.csv'
  path_ = path
  if GIT:
    path_ = git_path
  df = pd.read_csv(path_ + fname, encoding='ISO-8859-8')
  df.city = df.city.str.lower()
  df.city = df.city.str.translate(remove_punct_dict)
  LARGEST = df.city.tolist()
  df = df.set_index('city')
  return df, LARGEST
largest_df, LARGEST = load_largest()

d(largest_df.head())
p(largest_df.loc['toronto'].country)

<IPython.core.display.Javascript object>

,country
city,
tokyo,Japan
delhi,India
shanghai,China
so paulo,Brazil
ciudad de mxico mexico city,Mexico


Canada


In [ ]:
#nlp = en_core_web_md.load()
def update_nlp():
  matcher = PhraseMatcher(nlp.vocab)
  CITIES = set(CITIES_IL + CITIES_API_city)
  matcher.add("CITIES", None, *list(nlp.pipe(CITIES)))
  # country codes causes problems becuae many are similar to regular words (eg. 'IN' similar to 'in')
  #matcher.add("CITIES_API_country_code", None, *list(nlp.pipe(CITIES_API_country_code)))
  # due to duplicates with country codes ('IL' = Israel, and Ilinoie), we have to check manually for states
  #matcher.add("CITIES_API_state_code", None, *list(nlp.pipe(CITIES_API_state_code)))

  # Register the Span extension attributes "is_country" and "is_city" 
  check_country = lambda span: span.text.lower().translate(remove_punct_dict) in COUNTRIES
  Span.set_extension('is_country', getter=check_country, force=True)

  check_city = lambda span: span.text.lower().translate(remove_punct_dict) in CITIES
  Span.set_extension('is_city', getter=check_city, force=True)

  def gpe_component(doc):
      # Create an entity Span with the label "GPE" for all matches
      matches = matcher(doc)
      doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches]
      return doc

  # Add the component to the pipeline
  pipes = nlp.pipe_names
  if 'gpe_component' in pipes:
    nlp.remove_pipe('gpe_component')

  nlp.add_pipe(gpe_component, first=True)
  #print(nlp.pipe_names)

update_nlp()

## Get Parts (POS)

In [ ]:
def get_parts(text, disp=False, verbose=True):
  global DEBUG
  doc = nlp(text.lower().translate(remove_punct_dict))
  date, gpe, ents = [], {}, []
  action = ''
  multiple_cities = False
  if disp: p()
  for token in doc:
    if disp:
      p(f'{token.text}, dep_={token.dep_}, ent_type={token.ent_type_}, head={token.head}, lemma_={token.lemma_}, pos_={token.pos_}, tag_={token.tag_}')
    if token.ent_type_ == 'GPE': ents.append((token.text.lower(), token))
  for entity in doc.ents:
      if disp:
        print(f"{entity.text}, {entity.label_}='{spacy.explain(entity.label_)}', is_city={entity._.is_city}, is_country={entity._.is_country}")
      ent_text = entity.text.lower().translate(remove_punct_dict)
      if entity.label_ == 'DATE': date.append(ent_text)
      if entity.label_ == 'GPE':
        token = None
        for e in ents:
          if ent_text == e[0]:
            token = e[1] 
            break
        # verify the GPE is a true GPE for cases that a city has a name that has another meaninig
        if token and token.text != token.lemma_: # a true GPE cannot be lemmatized
          continue
        # for example there is a city in Denemark that has a name 'rain'
        if token and len(ents) > 1 and (token.head == token or token.pos_ not in ['PROPN', 'PRON', 'NOUN']) and not entity._.is_country:
          valid_gpe = False
          for e in ents:
            if ent_text != e[0]:
              tk = e[1]
              if tk.head == token:  # the token is connected to another gpe, so it is a valid gpe (for example, (paris, canada) are connected
                valid_gpe = True
                break
          if not valid_gpe:
            if DEBUG: p('A:', ent_text)
            continue
        ent_type = 'state'
        if DEBUG: p('0:', ent_text, entity._.is_city, entity._.is_country)
        if entity._.is_city and not entity._.is_country: # 'canada' is a country and also a city in Portugal
          ent_type = 'city'
          if ent_text in df_CITIES_API.index.values: 
            code_country = df_CITIES_API.loc[ent_text].country
            if type(code_country) != str:
              multiple_cities = True
              ent_country = countries_df.loc[code_country[0]].Name
            else:
              ent_country = countries_df.loc[code_country].Name
          elif ent_text in CITIES_IL:
            ent_country = 'israel'
          else:
            ent_country = ''
        elif entity._.is_country or ent_text in CITIES_API_country_code:
          ent_type = 'country'

        if ent_type not in gpe:
          gpe[ent_type] = []
        if ent_type == 'city':
           gpe[ent_type].append((entity.text, ent_country))
        else:
          gpe[ent_type].append(entity.text)
  if len(date) == 0:
    date = 'current'

  if DEBUG: p('1:', multiple_cities, gpe)
  if multiple_cities or ('city' in gpe and len(gpe['city']) > 1):
    if 'country' in gpe:
      # verify country code of the city is correct, when there multiple cities
      for i, (city, city_country) in enumerate(gpe['city']):
        country = gpe['country'][0]
        if city_country != country:
          for code_country in df_CITIES_API.loc[city].country:
            if city_country == countries_df.loc[code_country].Name:
              # found a matching country
              gpe['city'][i] = (city, country)
              break
    else:
      check_capital = True
      if len(gpe['city']) == 2: # maybe enter a country which has city with that name (eg. Toronto, US = there is a city name 'us' in france and 'usa' in japan)
        # 'city': [('toronto', 'Canada'), ('us', 'france')]
        for i in range(1,-1,-1):
          city = gpe['city'][i][0].upper()
          if city in countries_df.index: #usually the 2nd "city" is the country
            city_country = countries_df.loc[city].Name
            j = 0 if i == 1 else 1
            gpe['city'] = (gpe['city'][j][0], city_country)
            gpe['country'] = city_country
            check_capital = False
            break
      if check_capital:
        # check if one of the cities is a capital or a large city, then use it
        for i, (city, city_country) in enumerate(gpe['city']):
          if city in capitals_df.index.values:
            # found a matching country
            city_country = capitals_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
          if city in largest_df.index.values:
            # found a matching country
            city_country = largest_df.loc[city].country
            gpe['city'][i] = (city, city_country)
            break
  elif 'country' in gpe and 'city' in gpe:  # country with city, verify correct
      # verify country code of the city is correct, when user entered 
      city, city_country = gpe['city'][0]
      country = gpe['country'][0]
      if city_country != country:
        gpe['city'] = [(city, f'ERROR: city [{city}] not found in country [{country}] - try a different spelling of the city')]
  elif 'country' in gpe and 'city' not in gpe:  # country without city, get the capital
    country = gpe['country'][0]
    if country in CAPITALS:
      gpe['city'] = [(CAPITALS[country], country)]
  result = {}
  result['date'] = date
  result.update(gpe)
  if disp:
    p(result)
  elif verbose and 'city' not in result:
    p(f'FAILED: org [{text}], {result}')
  return result

DEBUG = False
if False and TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("Today weather in a Ra'anana")  
  get_parts("Today weather in a Raanana")
  get_parts("weather in toronto") # there is a city toronto in USA and toronto is not the capital of Canada so we take the first city in the db
  get_parts("weather in toronto, canada")
  get_parts("weather paris next 5 days")
  get_parts("tell me how is the weather in the ra- anana in the")
  get_parts("rain in toronto")
get_parts("weather in toronto, us")
get_parts("weather in toronto, canada")
get_parts("weather in jenin")
get_parts("weather in jenin, israel")


org [tell me how is the weather in the ra- anana in the], {'date': 'current'}


In [ ]:
get_parts("please tell me if it will rain tonight in Rain")

org [please tell me if it will rain tonight in Rain], {'date': 'current'}


{'date': 'current'}

In [ ]:
DEBUG = False
get_parts("tomorrow and today rain in spain")
get_parts("tomorrow rain in madrid")
get_parts("tomorrow rain in beersheba")
get_parts("tomorrow rain in israel")
get_parts("weather paris Canada")
get_parts("weather paris, Canada")
get_parts("tomorrow rain in yafo")

{'city': [('yafo', 'israel')], 'date': ['tomorrow']}

In [ ]:
if TEST:
  get_parts("Today weather in a Ra-anana")
  get_parts("weather in tel aviv")
  get_parts("tomorrow weather in haifa")
  get_parts("tomorrow rain in beersheba")
  get_parts("weather paris today and tomorrow")
  get_parts("weather paris")
  get_parts("weather paris in December")

In [ ]:
def chatbot_response(text):
  return ""

In [ ]:
get_parts("ra'anana")
get_parts("Today weather in a Tel Aviv")
get_parts("weather paris")
get_parts("paris weather ")
get_parts('Paris weather in')
get_parts("weather shelomi")

{'city': [('shelomi', 'israel')], 'date': 'current'}

## Run the bot

In [ ]:
nltk.download(['punkt', 'stopwords'])
stopwords = nltk.corpus.stopwords.words('english')

<IPython.core.display.Javascript object>

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<IPython.core.display.Javascript object>

In [ ]:
get_parts("weather in tel-aviv")
get_parts("weather in TelAviv")

org [weather in tel-aviv], {'date': 'current'}
org [weather in TelAviv], {'date': 'current'}


{'date': 'current'}

## Google search

In [ ]:
import bs4
def get_google(text):
  url = "https://google.com/search?q=" + text + '&hl=en'
  request_result = requests.get( url )
  soup = bs4.BeautifulSoup(request_result.text, "html.parser")
  spans = soup.find_all('span', class_='BNeawe')
  new_text = ''
  for span in spans: 
    if isinstance(span.contents[0], bs4.NavigableString):
      new_text += str(span.contents[0]) + ' '
  return new_text
get_google("weather ra anana")

"Ra'anana, Israel  /  Weather "

> when similarity was .075 it matched the sentence

>"tell me how is the weather in the ra- anana in the"

>to "what's up" of tag ["greeting"] so we increased it to 0.85

In [ ]:
do = True
GREETING, NOANSWER = 0, 3
p(intents['intents'][GREETING]['responses'][0])
min_similarity = 0.84
VERBOSE = 0

while do:
  user_msg = input().lower().strip()
  if user_msg == '':
    answers = intents['intents'][NOANSWER]['responses']
    p("WeatherBot: " + random.choice(answers))
    continue
  round = 0
  answer = ''
  org_msg = user_msg
  while round < 3 and do:
    if VERBOSE:   p(f'round {round}: {user_msg}')
    doc1 = nlp(user_msg)
    answers, tag = None, None
    top_similarity = 0
    # verify not an erroeous finding (if we have GPE in the doc we assume it is not a general request)
    if len([ent for ent in doc1.ents if ent.label_ == 'GPE']) == 0:
      for ints in intents['intents']:
        #p(ints)
        for text in ints['patterns']:
          doc2 = nlp(text)
          # Get the similarity of doc1 and doc2
          similarity = doc1.similarity(doc2)
          if VERBOSE > 2: p(f"{similarity}, {doc2}, {[ints['tag']]}")
          if similarity > top_similarity:
            top_similarity = similarity
            answers = ints['responses']
            tag = ints['tag']
    #  p(top_similarity, tag)
    #  p(answers)
      if top_similarity > min_similarity:
        if tag == "goodbye":
          do = False
        if tag in ["goodbye", "greeting", "thanks", "options"]:
          answer = random.choice(answers)
          break
    parts = get_parts(user_msg, VERBOSE > 1, False)
    if VERBOSE:  p(parts)
    if 'date' in parts and 'city' in parts and 'ERROR' not in parts['city'][0][1]:
      #p(parts['city'][0][0])
      answer = get_weather(parts['date'], parts['city'], action=None)
    elif 'city' in parts and 'ERROR' in parts['city'][0][1]:
      answer = parts['city'][0][1]
    elif round == 0:
      round = 1
      user_msg = ''
      space = ' '
      for token in doc1:
        if token.text in stopwords: continue
        text = token.text.lower().translate(remove_punct_dict)
        if text == '': continue
        if text[-1] == '-': # take care of 'ra- anana'
          user_msg += space + text[:-1]
          space = ''
        else:
          user_msg += space + text
          space = ' '
      continue
    elif round == 1:
      round = 2
      user_msg = get_google(org_msg)
      if user_msg != '':
        continue
    break
  if answer == '': answer = "You need to tell me a city to check (maybe spell in differently)"
  p(f"WeatherBot: {answer}")
# weather in jenin, israel  

Hi. My name is WeatherBot and I'm a chatbot. If you want to exit, type 'Bye' If you want help type 'Help'
weather toronto
{'date': 'current', 'city': [('toronto', 'Canada')]}
WeatherBot: In Toronto, Canada the current weather is: clear sky
bye
WeatherBot: Bye! Come back again soon.
